# Import 

In [1]:
# 의류 데이터 분류하는 모델 만들어보기 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf # 텐서플로우 import 
import numpy as np
import matplotlib.pyplot as plt


# 데이터 불러오기 & 전처리

In [2]:
(x_train_all, y_train_all), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data() # 데이터 불러오기. 튜플로 묶어서 반환

In [3]:
# 훈련 세트와 검증 세트로 분류. train_test_split() 사용
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train_all, y_train_all, stratify = y_train_all, test_size = 0.2, random_state = 42) # random_state는 안해도 되는데 예제랑 같다는 걸 보여주기 위해 사용한 것. 20%(test_size = 0.2)를 검증 세트로 나눈다.

In [4]:
# 입력 데이터 정규화. 0~1사이 값으로 나타내기 위해 255로 나눠줌 
x_train = x_train / 255
x_val = x_val / 255

In [5]:
# 28*28를 784(28*28하면 나오는거)로 변경. 왜냐하면 내가 설계한 모델은 1차원 배열을 요구하기 때문.
#한 줄로 쫙 이어붙혔다고 보면 된다.
x_train = x_train.reshape(-1, 784)
x_val = x_val.reshape(-1,784)

In [6]:
# 정답 데이터들을 원-핫 인코딩 처리
y_train_encoded = tf.keras.utils.to_categorical(y_train)
y_val_encoded = tf.keras.utils.to_categorical(y_val)

### 텐서플로우로 모델 생성(without keras)

In [7]:
# 가중치, 절편 초기화
w = tf.Variable(tf.zeros(shape =(1)))
b = tf.Variable(tf.zeros(shape =(1)))

# 경사 하강법 옵티마이저 설정(=학습법 설정)
optimizer = tf.optimizers.SGD(lr = 0.01) # 학습률 0.01의 확률적 경사 하강법(하나씩 훈련)으로 훈련시킬 것
#에포크 횟수 설정
num_epochs = 10
for step in range(num_epochs):
    #자동 미분을 위해 연산 과정 기록
    with tf.GradientTape() as tape:
        z_net = w * x_train + b # 출력값
        z_net = tf.reshape(z_net, [-1]) 
        sqr_errors = tf.square(y_train - z_net) #오차 제곱을 손실 함수로 사용
        mean_cost = tf.reduce_mean(sqr_errors)
    # 손실 함수에 대한 가중치의 그래디언트 계산
    grads = tape.gradient(mean_cost, [w,b])
    # 옵티마이저에 그레디언트 반영(가중치, 절편 값 수정. -> 학습)
    optimizer.apply_gradients(zip(grads, [w,b]))

# 에러 뜬다. 왜지? 하씨벌...모르겠다. 

InvalidArgumentError: Incompatible shapes: [48000] vs. [37632000] [Op:Sub]

### keras를 이용 

In [10]:
# 모델 생성
model = tf.keras.models.Sequential() # Sequential 클래스로 모델 생성. Sequential 클래스는 '순차적으로 레이어를 쌓은 신경망 모델
# 완전 연결층(Dense Layer) 추가
model.add(tf.keras.layers.Dense(1))
# 옵티마이저(학습법), 손실 함수 지정
model.compile(optimizer = 'sgd', loss = 'mse')
# 훈련 데이터를 사용하여 에포크 횟수만큼 훈련
model.fit(x_train, y_train, epochs = 10)

# 코드 양이 확 줄었음.
# 케라스는 다른 라이브러리에서도 사용 가능. 텐서플로우는 케라스를 주력 파이썬 API로 지정함. tensorflow.keras는 케라스 명세를 따르는 텐서플로우만을 위한 구현임.
# 위 코드는 은닉층, 출력층을 Dense 클래스의 객체(Dense Layer)로 구성하고 이를 Sequential 객체(모델)에 추가한 것.
# 마치 층을 쌓으며 모델을 만드는 것 같아 직관적으로 인공신경망의 층을 설계할 수 있다고 말함. 앞서 한 클래스에 가중치, 절편을 다 때려넣은 MultiClassNetwork와 비교해보면 정말 직관적이라는 사실을 알 수 있음.

Epoch 1/10
1500/1500 [==============================] - 1s 439us/step - loss: nan
Epoch 2/10
1500/1500 [==============================] - 1s 416us/step - loss: nan
Epoch 3/10
1500/1500 [==============================] - 1s 441us/step - loss: nan
Epoch 4/10
1500/1500 [==============================] - 1s 447us/step - loss: nan
Epoch 5/10
1500/1500 [==============================] - 1s 423us/step - loss: nan
Epoch 6/10
1500/1500 [==============================] - 1s 466us/step - loss: nan
Epoch 7/10
1500/1500 [==============================] - 1s 397us/step - loss: nan
Epoch 8/10
1500/1500 [==============================] - 1s 384us/step - loss: nan
Epoch 9/10
1500/1500 [==============================] - 1s 484us/step - loss: nan
Epoch 10/10
1500/1500 [==============================] - 1s 449us/step - loss: nan


## Sequential 클래스 사용 방법

In [ ]:
# 완전 연결 신경망을 만들려면 Sequential와 Dense를 함께 사용해야함. Sequential를 초기화할 때 층을 추가찰 수도 있고 빈 공간으로 초기화 후 add()로 층을 추가하는 방법이 있음.
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

# model = Sequential([Dense(...), ...]) # 방법 1. 초기화할 때 Dense Layer를 추가

# 빈 모델로 초기화 후 add로 Layer추가
# model.Sequential()
# dense = Dense(...)

## Dense 클래스의 사용 방법

In [ ]:
# unit : 뉴런의 개수
# activation : 활성화 함수
# Dense(unit = 100, activation = 'sigmoid') <- 작성 예. 활성화 함수는 softmax, tanh, relu 등 종류가 많다. 
# keras에서는 가중치를 커널(kernel)이라고 부른다.(커널이랑 가중치랑 의미가 같으니 혼동하지 않기) 가중치를 규제하기 위해 kernel_initializer 매개변수가 존재한다. 

## 모델의 최적화 알고리즘과 손실 함수를 지정

In [ ]:
# 모델을 훈련하기 위해서는 최적화 알고리즘이나 손실 함수 지정
# 최적화 알고리즘 : 경사 하강법 알고리즘 
# 손실 함수 : 크로스 엔트로피 손실 함수

# optimizer : 최적화 알고리즘 지정. compile() 함수에서 최적화 알고리즘과 손실 함수를 지정한다. 
# loss : 손실 함수. 제곱 오차(정답값과 예측값 차이(=오차값)을 제곱한거는 mse, 로지스틱 손실 함수는 binary_crossentropy, 다중 분류 신경망인 경우에는 cartegorical_crossentropy다.
# model.compile(optimizer = 'sgd', loss = 'cartegorical_crossentropy') <- 기본 경사 하강법으로 'sgd' 사용. sgd = 확률적 경사 하강법(데이터 하나씩 학습). 손실 함수는 다중 분류 모델을 사용한다고 하면 cartegorical_crossentropy.
# 학습률의 기본값은 0.01

## 모델의 훈련과 예측

In [ ]:
# Sequential 클래스의 fit()로 훈련, predict()로 예측.
# 검증은 evaluate()를 사용.

# 전형적인 Sequential 클래스의 사용 방법(데이터는 전처리 했다고 가정 - 당연히 사용 용도에 맞게 전처리 해놔야함)

model = Sequential() # 모델 초기화
# 레이어 추가
model.add(Dense(...)) # Dense Layer를 추가하는데 이 때 따로 Dense Layer를 선언하기 귀찮으니 add 함수 안에서 Dense Layer를 바로 만들어서 넣어줌. 
model.add(Dense(...)) 
# 최적화 알고리즘, 손실 함수 지정
model.compile(optimizer='...', loss='...')
# 훈련, 예측, 평가
model.fit(x, y, epochs = ...) # 훈련
model.predict(x) # 예측
model.evaluate(x, y) # 평가
